<a href="https://colab.research.google.com/github/Untick/Usedesk/blob/main/Lapkov%20Anatoly/a_lapkov_usedesk_ru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import os
import requests
import pandas as pd

from IPython.display import display

In [5]:
FILE_1_ID = '1PwMWQUCETRusiWN9hep2-12GWkdBrpkR'
FILE_1_NAME = 'file_1.xlsx'
FILE_2_ID = '1cCjgzBbSsBnzEfRQ9NgIv_ZVjmRywoqV'
FILE_2_NAME = 'file_2.xlsx'

In [6]:
def get_confirm_token(response):
  for key, value in response.cookies.items():
    if key.startswith('download_warning'):
      return value
  return None

def save_response_content(response, destination):
  CHUNK_SIZE = 32768
  with open(destination, 'wb') as f:
    for chunk in response.iter_content(CHUNK_SIZE):
      if chunk:
        f.write(chunk)

def download_file_from_google_drive(id, destination):
  URL = 'https://docs.google.com/uc?export=download'
  session = requests.Session()
  response = session.get(URL, params = { 'id' : id }, stream = True)
  token = get_confirm_token(response)
  if token:
    params = { 'id' : id, 'confirm' : token }
    response = session.get(URL, params = params, stream = True)
  save_response_content(response, destination)    


In [7]:
download_file_from_google_drive(FILE_1_ID, FILE_1_NAME)
download_file_from_google_drive(FILE_2_ID, FILE_2_NAME)

In [11]:
file_1_data = pd.read_excel(FILE_1_NAME)
file_2_data, file_2_links = pd.read_excel(FILE_2_NAME, sheet_name = [0, 2]).values()

In [12]:
display(file_1_data)
display(file_2_data)
display(file_2_links)

,#NAME?,#NAME?.1,company_id,рейтинг,месяц,всего,есть ответ,выполнен,закрыт,nps,...,extreme,агент+,агент-,каналы+ all,каналы+ p,каналы- all,каналы- p,правила_cnt,nps_cnt,nps_avg
0,#NAME?,NaN,1,1,1,1110,146,40,124,4,...,0,0,14,13,1,35,2,0,NaN,NaN
1,#NAME?,NaN,1,1,2,2615,416,54,672,101,...,4,5,2,60,25,79,12,0,NaN,NaN
2,#NAME?,NaN,1,1,3,4226,639,167,625,140,...,2,11,22,101,10,100,21,0,NaN,NaN
3,#NAME?,NaN,1,1,4,3484,351,61,685,158,...,3,5,3,42,8,48,4,0,NaN,NaN
4,#NAME?,NaN,1,1,5,3950,620,117,559,51,...,13,2,0,63,9,56,17,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2251,#NAME?,NaN,376,2,2,28074,27996,28066,1,0,...,1,0,0,0,0,0,0,0,1.0,1.0
2252,#NAME?,NaN,376,2,3,6330,6297,6299,22,0,...,0,3,1,1,0,1,0,0,1.0,1.0
2253,#NAME?,NaN,376,2,4,35179,34848,17682,17485,0,...,0,0,1,1,0,0,0,0,1.0,1.0
2254,#NAME?,NaN,376,2,5,7735,6235,383,5854,0,...,0,4,0,3,0,3,1,0,NaN,NaN


,#NAME?,comp_id_new,месяц_desc,дата,рейтинг,всего,есть ответ,выполнен,закрыт,nps,...,u-task,u_problem,u_incident,u_low,u_medium,u_urgent,u_extreme,licens,бз,бз_ст
0,#NAME?,1,1,2023-06-01,2,3709,3442,578,2466,1231,...,0,0,0,0,2,0,0,13,1,4
1,#NAME?,1,2,2023-05-01,2,5573,5301,36,4715,2004,...,0,0,0,0,5,0,0,13,1,4
2,#NAME?,1,3,2023-04-01,2,6138,5983,4,5496,2242,...,0,0,0,0,1,0,0,13,1,4
3,#NAME?,1,4,2023-03-01,2,7834,7664,1,7094,2851,...,0,0,0,0,6,0,0,13,1,4
4,#NAME?,1,5,2023-02-01,2,7344,7252,1,6657,2686,...,0,0,0,0,2,0,0,13,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11774,#NAME?,465,1,2023-05-01,1,11,11,0,0,0,...,0,0,0,0,0,0,0,10,0,0
11775,#NAME?,466,1,2023-06-01,1,88,12,0,84,0,...,0,0,0,0,2,0,0,9,0,0
11776,#NAME?,466,2,2023-05-01,1,7,7,0,0,0,...,0,0,0,0,0,0,0,9,0,0
11777,#NAME?,467,1,2023-06-01,1,57,55,0,25,1,...,0,0,0,0,2,0,0,3,1,1


,comp_id_new,comp_id_old
0,1,2
1,2,3
2,3,5
3,4,0
4,5,6
...,...,...
463,464,0
464,465,0
465,466,0
466,467,0


In [14]:
print(file_1_data.dtypes)
print(file_2_data.dtypes)
print(file_2_links.dtypes)

#NAME?                     object
#NAME?.1                  float64
company_id                  int64
рейтинг                     int64
месяц                       int64
всего                       int64
есть ответ                  int64
выполнен                    int64
закрыт                      int64
nps                         int64
email                       int64
telephony                   int64
api                         int64
usedesk_chat                int64
vk                          int64
telegram                    int64
whatsapp                    int64
fb_instagram                int64
chat                        int64
instagram                   int64
facebook                    int64
pact_telegram_personal      int64
`twitter-mention`           int64
chat2desk                   int64
slack_thread                int64
`twitter-direct`            int64
question                    int64
task                        int64
problem                     int64
incident      